# **Sub-Graphs in LangGraph**

In **LangGraph**, sub-graphs provide a powerful way to manage complex workflows by breaking them into smaller, modular components. Instead of dealing with a single, large graph containing numerous nodes and multiple routes, sub-graphs allow you to organize your logic in a cleaner and more maintainable structure.

## **Why Use Sub-Graphs?**

- **Modularity**: Simplifies the design by splitting the graph into smaller, reusable components.
- **Parallel Execution**: Allows for executing multiple sub-graphs simultaneously when their tasks are independent.
- **Conditional Traversal**: Enables dynamic navigation between sub-graphs based on specific conditions.

## **How Sub-Graphs Work**

1. **Definition**:
   - Each sub-graph is a smaller graph with its own nodes, edges, and logic.
   - Sub-graphs can handle isolated tasks or workflows.

2. **Integration**:
   - Sub-graphs can be embedded in a parent graph.
   - Data can flow between the parent graph and sub-graphs through shared states.

3. **Execution Options**:
   - **Parallel Execution**: Run multiple sub-graphs at the same time for independent workflows.
   - **Conditional Edges**: Navigate through sub-graphs based on conditions or decision logic.

## **Benefits of Sub-Graphs**

- **Scalability**: Breaks down large, complex graphs into manageable units.
- **Reusability**: Sub-graphs can be reused in different contexts or workflows.
- **Flexibility**: Easily add, modify, or remove sub-graphs without affecting the entire graph.
- **Efficiency**: Parallel execution improves performance for independent tasks.

Sub-graphs make LangGraph workflows more structured and easier to debug while maintaining flexibility for complex applications.


## **Types of Sub-Graphs**

There are two main types of sub-graphs in LangGraph, based on how they interact with the parent graph:

1. **Shared Schema Keys**:  
   - The parent graph and sub-graph share at least one same schema keys (There is a common key between Parent State and sub_graph State).  
   - In this case, you can directly add a node with the compiled sub-graph.  

2. **Different Schemas**:  
   - The parent graph and sub-graph have different schemas (There is no common key between Parent State and sub_graph State).  
   - Here, you need to use a node function to invoke the sub-graph. This allows you to transform the state before or after calling the sub-graph, ensuring compatibility between the two schemas.


## **1- Shared Schema Key**

### Let's Build This simple Sub-Graph in main Graph


<p align="center">
  <img src="image_1.png" alt="Explain Graph">
</p>


In [1]:
%%capture --no-stderr
%pip install -U langgraph

In [ ]:
from langgraph.graph import START, StateGraph,END
from typing import TypedDict,List,Annotated
import operator


# Define subgraph
class SubgraphState(TypedDict):
    parent_key: str  # note that this key is shared with the parent graph state (anything will be added here will automatically be added in same key at parent state)
    sub_graph_messages: Annotated[List[str], operator.add] # accumalting our messages here

# node 1 inside sub-graph
def subgraph_node_1(state: SubgraphState):
    print("state at subgraph_node_1",state)
    return {"sub_graph_messages": ["Node 1 in sub-graph"]}

# node 2 inside sub-graph
def subgraph_node_2(state: SubgraphState):
    # parent_key is updated with data to be shared with parent_key
    print("state at subgraph_node_2",state)
    return {"sub_graph_messages": ["Node 2 in sub-graph"]}

# node 3 inside sub-graph
def subgraph_node_3(state: SubgraphState):
    # parent_key is updated before leaving graph (automatically would update parent_key in orginal parent state)
    print("state at subgraph_node_3 we update parent before leaving graph only",state)
    # parent_key is considered global here in our overall graph thus it would include data acummlated in parent graph before entering sub-graph
    print("parent_key before adding anything : ",state["parent_key"] )
    return {"parent_key": state["parent_key"] + ","+ state["sub_graph_messages"][0] +","+ state["sub_graph_messages"][1]}

# lets build our sub-graph
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_node(subgraph_node_3)

subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph_builder.add_edge("subgraph_node_2", "subgraph_node_3")
subgraph_builder.add_edge("subgraph_node_3", END)

subgraph = subgraph_builder.compile()

# Let's build our parent graph
class ParentState(TypedDict):
    parent_key: str


def node_1(state: ParentState):
    return {"parent_key": state["parent_key"]}

builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
# note that we're adding the compiled subgraph as a node to the parent graph
builder.add_node("node_2", subgraph)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", END)

graph = builder.compile()

In [3]:
input_state = {"parent_key": "Here We Go !"}
output = graph.invoke(input_state)

state at subgraph_node_1 {'parent_key': 'Here We Go !', 'sub_graph_messages': []}
state at subgraph_node_2 {'parent_key': 'Here We Go !', 'sub_graph_messages': ['Node 1 in sub-graph']}
state at subgraph_node_3 we update parent before leaving graph only {'parent_key': 'Here We Go !', 'sub_graph_messages': ['Node 1 in sub-graph', 'Node 2 in sub-graph']}
parent_key before adding anything Here We Go !


In [4]:
# After updating parent_key / we notice there is no sub_graph_messages
output

{'parent_key': 'Here We Go !,Node 1 in sub-graph,Node 2 in sub-graph'}

#### Conclusion
- Parent Key here is considered a Global key among our states that we have built
- Notice we didn't invoke our sub-graph we just added it as a node

### Let's Build This Advanced Sub-Graph in main Graph


<p align="center">
  <img src="image_2.png" alt="Explain Graph">
</p>


**Note** what is being shared or in other words Global here is sub_graph_1_messages and sub_graph_2_messages not parent_key also we are going to pass through both sub graph 1 and sub graph 2 at the same time (in parallel)

#### **First Sub-Graph**

In [5]:
from langgraph.graph import START, StateGraph,END
from typing import TypedDict,List,Annotated
import operator


# Define subgraph
class SubgraphState_1(TypedDict):
    sub_graph_1_messages: Annotated[List[str], operator.add]

# node 1 inside sub-graph
def subgraph_node_1(state: SubgraphState_1):
    print("state at subgraph_node_1",state)
    return {"sub_graph_1_messages": ["Node 1 in sub-graph 1"]}

# node 2 inside sub-graph
def subgraph_node_2(state: SubgraphState_1):
    # parent_key is updated with data to be shared with parent_key
    print("state at subgraph_node_2",state)
    return {"sub_graph_1_messages": ["Node 2 in sub-graph 1"]}


subgraph_builder = StateGraph(SubgraphState_1)
subgraph_builder.add_node("subgraph_1_node_1",subgraph_node_1)
subgraph_builder.add_node("subgraph_1_node_2",subgraph_node_2)

subgraph_builder.add_edge(START, "subgraph_1_node_1")
subgraph_builder.add_edge("subgraph_1_node_1", "subgraph_1_node_2")
subgraph_builder.add_edge("subgraph_1_node_2", END)

subgraph_1= subgraph_builder.compile()

#### **Sec. Sub-Graph**

In [6]:
# Define subgraph
class SubgraphState_2(TypedDict):
    sub_graph_2_messages: Annotated[List[str], operator.add]

# node 1 inside sub-graph
def subgraph_node_1(state: SubgraphState_2):
    print("state at subgraph_node_1",state)
    return {"sub_graph_2_messages": ["Node 1 in sub-graph 2"]}

# node 2 inside sub-graph
def subgraph_node_2(state: SubgraphState_2):
    # parent_key is updated with data to be shared with parent_key
    print("state at subgraph_node_2",state)
    return {"sub_graph_2_messages": ["Node 2 in sub-graph 2"]}


subgraph_builder = StateGraph(SubgraphState_2)
subgraph_builder.add_node("subgraph_2_node_1",subgraph_node_1)
subgraph_builder.add_node("subgraph_2_node_2",subgraph_node_2)

subgraph_builder.add_edge(START, "subgraph_2_node_1")
subgraph_builder.add_edge("subgraph_2_node_1", "subgraph_2_node_2")
subgraph_builder.add_edge("subgraph_2_node_2", END)


subgraph_2 = subgraph_builder.compile()

In [7]:
# Define parent graph
class ParentState(TypedDict):
    parent_key: str
    sub_graph_1_messages: Annotated[List[str], operator.add]
    sub_graph_2_messages: Annotated[List[str], operator.add]


def node_1(state: ParentState):
    return {"parent_key": state["parent_key"]}


builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
# note that we're adding the compiled subgraph as a node to the parent graph
builder.add_node("node_2", subgraph_1)
builder.add_node("node_3", subgraph_2)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_1", "node_3")

graph = builder.compile()

In [8]:
input_state = {"parent_key": "Here We Go !"}
# what's excuted first is returned first but they are excuted in parallel
output = graph.invoke(input_state)

state at subgraph_node_1 {'sub_graph_1_messages': []}
state at subgraph_node_2 {'sub_graph_1_messages': ['Node 1 in sub-graph 1']}
state at subgraph_node_1 {'sub_graph_2_messages': []}
state at subgraph_node_2 {'sub_graph_2_messages': ['Node 1 in sub-graph 2']}


In [9]:
output

{'parent_key': 'Here We Go !',
 'sub_graph_1_messages': ['Node 1 in sub-graph 1', 'Node 2 in sub-graph 1'],
 'sub_graph_2_messages': ['Node 1 in sub-graph 2', 'Node 2 in sub-graph 2']}

## **2- Different Schemas**

<p align="center">
  <img src="Screenshot 2024-12-04 114330.png" alt="Explain Graph">
</p>


In [ ]:
from langgraph.graph import START, StateGraph
from typing import TypedDict,List,Annotated
import operator

 
# Define subgraph
class SubgraphState(TypedDict):
    sub_graph_messages: Annotated[List[str], operator.add]

# node 1 inside sub-graph
def subgraph_node_1(state: SubgraphState):
    print("state at subgraph_node_1",state)
    return {"sub_graph_messages": ["Node 1 in sub-graph"]}

# node 2 inside sub-graph
def subgraph_node_2(state: SubgraphState):
    # parent_key is updated with data to be shared with parent_key
    print("state at subgraph_node_2",state)
    return {"sub_graph_messages": ["Node 2 in sub-graph"]}


subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)

subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")

subgraph = subgraph_builder.compile()

In [ ]:
# Define parent graph
class ParentState(TypedDict):
    message: Annotated[List[str], operator.add]

def node_1(state: ParentState):
    print("State including input only" , state)
    #  then we add here 
    return {"message": [state["message"]]}

# note instead of Adding Node direcly as we have done before in graph we invoke graph and take response and add it in message of parent state
def node_2(state: ParentState):
    print("State including Before invoking graph" , state)
    response = subgraph.invoke({"sub_graph_messages":["hello"]})
    return{"message":[response]}

builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
# note that we're adding the compiled subgraph as a node to the parent graph
builder.add_node("node_2", node_2)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
graph = builder.compile()

In [37]:
input_state = {"message": ["Here We Go !"]}
# what's excuted first is returned first but they are excuted in parallel
output = graph.invoke(input_state)

State including input only {'message': ['Here We Go !']}
State including Before invoking graph {'message': ['Here We Go !', ['Here We Go !']]}
state at subgraph_node_1 {'sub_graph_messages': ['hello']}
state at subgraph_node_2 {'sub_graph_messages': ['hello', 'Node 1 in sub-graph']}


In [38]:
output

{'message': ['Here We Go !',
  ['Here We Go !'],
  {'sub_graph_messages': ['hello',
    'Node 1 in sub-graph',
    'Node 2 in sub-graph']}]}

#### Conclusion
- we invoke our sub-graph in node 2 and we took response inorder not to be lost and we added it in our parent state